### Diaspora AP Demo
Adapted from https://github.com/globus/globus-jupyter-notebooks/blob/master/Automation_Using_Globus_Flows.ipynb

In [ ]:
"""Use Diaspora AP in Globus Flows."""

from __future__ import annotations

import json
import time
import uuid

import globus_sdk
import globus_sdk.scopes
from diaspora_event_sdk import Client as GlobusClient

# ID of this tutorial notebook as registered with Globus Auth
CLIENT_ID = 'f794186b-f330-4595-b6c6-9c9d3e903e47'

In [ ]:
# Do a native app authentication flow to get tokens that allow us
# to interact with the Globus Flows service

scopes = [
    'openid',
    'profile',
    'email',
    globus_sdk.FlowsClient.scopes.manage_flows,
    globus_sdk.FlowsClient.scopes.run_manage,
]
native_auth_client = globus_sdk.NativeAppAuthClient(CLIENT_ID)
native_auth_client.oauth2_start_flow(requested_scopes=scopes)
print(f'Login Here:\n\n{native_auth_client.oauth2_get_authorize_url()}')

auth_code = input('Authorization Code: ')
response = native_auth_client.oauth2_exchange_code_for_tokens(auth_code)

tokens = response.by_resource_server
print(json.dumps(tokens, indent=2))

flows_authorizer = globus_sdk.AccessTokenAuthorizer(
    access_token=tokens['flows.globus.org']['access_token'],
)
flows_client = globus_sdk.FlowsClient(authorizer=flows_authorizer)

In [ ]:
# Create an Auth client so we can look up identities
auth_authorizer = globus_sdk.AccessTokenAuthorizer(
    access_token=tokens['auth.globus.org']['access_token'],
)
ac = globus_sdk.AuthClient(authorizer=auth_authorizer)

# Get the user's primary identity
primary_identity = ac.oauth2_userinfo()
identity_id = primary_identity['sub']

print(f"Username: {primary_identity['preferred_username']}")
print(f'ID: {identity_id}')

### Select a Topic

In [ ]:
c = GlobusClient()
print("User's OpenID:", c.subject_openid)
topic = 'topic' + c.subject_openid[-12:]
print(c.register_topic(topic))
print(c.list_topics())
print('Topic to produce/consume:', topic)

### 2.1.1 Produce Messages to AP without msg keys

In [ ]:
action_url = 'https://diaspora-action-provider.ml22sevubfnks.us-east-1.cs.amazonlightsail.com/'

flow_definition1 = {
    'Comment': 'Publish messages to Diaspora Event Fabric',
    'StartAt': 'PublishMessages',
    'States': {
        'PublishMessages': {
            'Comment': 'Send messages to a specified topic in Diaspora',
            'Type': 'Action',
            'ActionUrl': action_url,
            'Parameters': {
                'action.$': '$.input.action',
                'topic.$': '$.input.topic',
                'msgs.$': '$.input.msgs',
            },
            'ResultPath': '$.PublishMessages',
            'End': True,
        },
    },
}

In [ ]:
flow_title = f'Diapora-AP-Flow-{str(uuid.uuid4())[:4]}'
flow = flows_client.create_flow(
    title=flow_title,
    definition=flow_definition1,
    # definition=flow_definition2,
    input_schema={},
)
flow_id = flow['id']
flow_scope = globus_sdk.SpecificFlowClient(flow_id).scopes.make_mutable('user')
print(f"Successfully created flow: '{flow_title} (ID: {flow_id})")
print(f'View the flow in the Web App: https://app.globus.org/flows/{flow_id}')

In [ ]:
if flow_id not in tokens:
    # Do a native app authentication flow and get tokens that
    # include the newly deployed flow scope
    native_auth_client = globus_sdk.NativeAppAuthClient(CLIENT_ID)
    native_auth_client.oauth2_start_flow(requested_scopes=flow_scope)
    print(f'Login Here:\n\n{native_auth_client.oauth2_get_authorize_url()}')

    # Authenticate and come back with your authorization code;
    # paste it into the prompt below.
    auth_code = input('Authorization Code: ')
    token_response = native_auth_client.oauth2_exchange_code_for_tokens(
        auth_code,
    )

    # Save the new token in a place where the flows client can retrieve it.
    tokens[flow_id] = token_response.by_resource_server[flow_id]

    # These are the saved scopes for the flow
    print(json.dumps(tokens, indent=2))

In [ ]:
flow_input1 = {
    'input': {
        'action': 'produce',
        'topic': topic,
        'msgs': [
            {'content': 'hello world1'},
            {'content': 'hello world2'},
            {'content': 'hello world3'},
        ],
    },
}

In [ ]:
# Get a client for the flow
specific_flow_authorizer = globus_sdk.AccessTokenAuthorizer(
    access_token=tokens[flow_id]['access_token'],
)
print(tokens[flow_id]['access_token'])
specific_flow_client = globus_sdk.SpecificFlowClient(
    flow_id=flow_id,
    authorizer=specific_flow_authorizer,
)

# Run the flow
# Set a descriptive label for this flow run
run_label = f"Diaspora AP Flow by {primary_identity['preferred_username']}"
run = specific_flow_client.run_flow(
    body=flow_input1,
    label=run_label,
    tags=['tutorial', 'diaspora'],
)

# Get run details
run_id = run['run_id']
run_status = run['status']
print('This flow can be monitored in the Web App:')
print(f'https://app.globus.org/runs/{run_id}')
print(f'Flow run started with ID: {run_id} - Status: {run_status}')

# Poll the Flow service to check on the status of the flow
while run_status == 'ACTIVE':
    time.sleep(5)
    run = flows_client.get_run(run_id)
    run_status = run['status']
    print(f'Run status: {run_status}')

# Run completed
print(json.dumps(run.data, indent=2))

### 2.2.1 Produce Messages to AP with a Single Key

In [ ]:
flow_definition2 = {
    'Comment': 'Publish messages to Diaspora Event Fabric',
    'StartAt': 'PublishMessages',
    'States': {
        'PublishMessages': {
            'Comment': 'Send messages to a specified topic in Diaspora',
            'Type': 'Action',
            'ActionUrl': action_url,
            'Parameters': {
                'action.$': '$.input.action',
                'topic.$': '$.input.topic',
                'msgs.$': '$.input.msgs',
                'keys.$': '$.input.keys',
            },
            'ResultPath': '$.PublishMessages',
            'End': True,
        },
    },
}

In [ ]:
flows_client.update_flow(flow_id, definition=flow_definition2, input_schema={})

In [ ]:
flow_input2a = {
    'input': {
        'action': 'produce',
        'topic': topic,
        'msgs': [
            {'content': 'hello world1'},
            {'content': 'hello world2'},
            {'content': 'hello world3'},
        ],
        'keys': 'my-key-123',
    },
}

In [ ]:
# Get a client for the flow
specific_flow_authorizer = globus_sdk.AccessTokenAuthorizer(
    access_token=tokens[flow_id]['access_token'],
)
print(tokens[flow_id]['access_token'])
specific_flow_client = globus_sdk.SpecificFlowClient(
    flow_id=flow_id,
    authorizer=specific_flow_authorizer,
)

# Run the flow
# Set a descriptive label for this flow run
run_label = f"Diaspora AP Flow by {primary_identity['preferred_username']}"
run = specific_flow_client.run_flow(
    body=flow_input2a,
    label=run_label,
    tags=['tutorial', 'diaspora'],
)

# Get run details
run_id = run['run_id']
run_status = run['status']
print('This flow can be monitored in the Web App:')
print(f'https://app.globus.org/runs/{run_id}')
print(f'Flow run started with ID: {run_id} - Status: {run_status}')

# Poll the Flow service to check on the status of the flow
while run_status == 'ACTIVE':
    time.sleep(5)
    run = flows_client.get_run(run_id)
    run_status = run['status']
    print(f'Run status: {run_status}')

# Run completed
print(json.dumps(run.data, indent=2))

### 2.3.1 Produce Messages to AP with a List of Keys

In [ ]:
flow_input2b = {
    'input': {
        'action': 'produce',
        'topic': topic,
        'msgs': [
            {'content': 'hello world1'},
            {'content': 'hello world2'},
            {'content': 'hello world3'},
        ],
        'keys': [
            'my-key-123',
            'my-key-456',
            'my-key-789',
        ],
    },
}

In [ ]:
# Get a client for the flow
specific_flow_authorizer = globus_sdk.AccessTokenAuthorizer(
    access_token=tokens[flow_id]['access_token'],
)
print(tokens[flow_id]['access_token'])
specific_flow_client = globus_sdk.SpecificFlowClient(
    flow_id=flow_id,
    authorizer=specific_flow_authorizer,
)

# Run the flow
# Set a descriptive label for this flow run
run_label = f"Diaspora AP Flow by {primary_identity['preferred_username']}"
run = specific_flow_client.run_flow(
    body=flow_input2b,
    label=run_label,
    tags=['tutorial', 'diaspora'],
)

# Get run details
run_id = run['run_id']
run_status = run['status']
print('This flow can be monitored in the Web App:')
print(f'https://app.globus.org/runs/{run_id}')
print(f'Flow run started with ID: {run_id} - Status: {run_status}')

# Poll the Flow service to check on the status of the flow
while run_status == 'ACTIVE':
    time.sleep(5)
    run = flows_client.get_run(run_id)
    run_status = run['status']
    print(f'Run status: {run_status}')

# Run completed
print(json.dumps(run.data, indent=2))

### 3.1.1: Consume All Messages since the Earliest Available Message

In [ ]:
flow_definition3 = {
    'Comment': 'Consume messages to Diaspora Event Fabric',
    'StartAt': 'ConsumeMessages',
    'States': {
        'ConsumeMessages': {
            'Comment': 'Receive messages from a specified topic in Diaspora',
            'Type': 'Action',
            'ActionUrl': action_url,
            'Parameters': {
                'action.$': '$.input.action',
                'topic.$': '$.input.topic',
            },
            'ResultPath': '$.ConsumeMessages',
            'End': True,
        },
    },
}

flows_client.update_flow(flow_id, definition=flow_definition3, input_schema={})

In [ ]:
flow_input3a = {
    'input': {'action': 'consume', 'topic': topic},
}

In [ ]:
# Get a client for the flow
specific_flow_authorizer = globus_sdk.AccessTokenAuthorizer(
    access_token=tokens[flow_id]['access_token'],
)
print(tokens[flow_id]['access_token'])
specific_flow_client = globus_sdk.SpecificFlowClient(
    flow_id=flow_id,
    authorizer=specific_flow_authorizer,
)

# Run the flow
# Set a descriptive label for this flow run
run_label = f"Diaspora AP Flow by {primary_identity['preferred_username']}"
run = specific_flow_client.run_flow(
    body=flow_input3a,
    label=run_label,
    tags=['tutorial', 'diaspora'],
)

# Get run details
run_id = run['run_id']
run_status = run['status']
print('This flow can be monitored in the Web App:')
print(f'https://app.globus.org/runs/{run_id}')
print(f'Flow run started with ID: {run_id} - Status: {run_status}')

# Poll the Flow service to check on the status of the flow
while run_status == 'ACTIVE':
    time.sleep(5)
    run = flows_client.get_run(run_id)
    run_status = run['status']
    print(f'Run status: {run_status}')

# Run completed
print(json.dumps(run.data, indent=2))

### 3.1.2: Consume Messages since a Timestamp

In [ ]:
flow_definition4 = {
    'Comment': 'Consume messages to Diaspora Event Fabric',
    'StartAt': 'ConsumeMessages',
    'States': {
        'ConsumeMessages': {
            'Comment': 'Receive messages from a specified topic in Diaspora',
            'Type': 'Action',
            'ActionUrl': action_url,
            'Parameters': {
                'action.$': '$.input.action',
                'topic.$': '$.input.topic',
                'ts.$': '$.input.ts',
            },
            'ResultPath': '$.ConsumeMessages',
            'End': True,
        },
    },
}

flows_client.update_flow(flow_id, definition=flow_definition4, input_schema={})

In [ ]:
flow_input4a = {
    'input': {'action': 'consume', 'topic': topic, 'ts': 1715930522000},
}

In [ ]:
# Get a client for the flow
specific_flow_authorizer = globus_sdk.AccessTokenAuthorizer(
    access_token=tokens[flow_id]['access_token'],
)
print(tokens[flow_id]['access_token'])
specific_flow_client = globus_sdk.SpecificFlowClient(
    flow_id=flow_id,
    authorizer=specific_flow_authorizer,
)

# Run the flow
# Set a descriptive label for this flow run
run_label = f"Diaspora AP Flow by {primary_identity['preferred_username']}"
run = specific_flow_client.run_flow(
    body=flow_input4a,
    label=run_label,
    tags=['tutorial', 'diaspora'],
)

# Get run details
run_id = run['run_id']
run_status = run['status']
print('This flow can be monitored in the Web App:')
print(f'https://app.globus.org/runs/{run_id}')
print(f'Flow run started with ID: {run_id} - Status: {run_status}')

# Poll the Flow service to check on the status of the flow
while run_status == 'ACTIVE':
    time.sleep(5)
    run = flows_client.get_run(run_id)
    run_status = run['status']
    print(f'Run status: {run_status}')

# Run completed
print(json.dumps(run.data, indent=2))

### 3.2.1: Consume Messages with Group ID

In [ ]:
flow_definition5 = {
    'Comment': 'Consume messages to Diaspora Event Fabric',
    'StartAt': 'ConsumeMessages',
    'States': {
        'ConsumeMessages': {
            'Comment': 'Receive messages from a specified topic in Diaspora',
            'Type': 'Action',
            'ActionUrl': action_url,
            'Parameters': {
                'action.$': '$.input.action',
                'topic.$': '$.input.topic',
                'group_id.$': '$.input.group_id',
            },
            'ResultPath': '$.ConsumeMessages',
            'End': True,
        },
    },
}

flows_client.update_flow(flow_id, definition=flow_definition5, input_schema={})

In [ ]:
flow_input5a = {
    'input': {
        'action': 'consume',
        'topic': topic,
        'group_id': 'my-group-1234',
    },
}
print(flow_input5a)

In [ ]:
# Get a client for the flow
specific_flow_authorizer = globus_sdk.AccessTokenAuthorizer(
    access_token=tokens[flow_id]['access_token'],
)
print(tokens[flow_id]['access_token'])
specific_flow_client = globus_sdk.SpecificFlowClient(
    flow_id=flow_id,
    authorizer=specific_flow_authorizer,
)

# Run the flow
# Set a descriptive label for this flow run
run_label = f"Diaspora AP Flow by {primary_identity['preferred_username']}"
run = specific_flow_client.run_flow(
    body=flow_input5a,
    label=run_label,
    tags=['tutorial', 'diaspora'],
)

# Get run details
run_id = run['run_id']
run_status = run['status']
print('This flow can be monitored in the Web App:')
print(f'https://app.globus.org/runs/{run_id}')
print(f'Flow run started with ID: {run_id} - Status: {run_status}')

# Poll the Flow service to check on the status of the flow
while run_status == 'ACTIVE':
    time.sleep(5)
    run = flows_client.get_run(run_id)
    run_status = run['status']
    print(f'Run status: {run_status}')

# Run completed
print(json.dumps(run.data, indent=2))

### 3.3.1 Consume Messages with Filters (a msg is returned if filter1 OR filter2 is met; within a filter, all conditions must be met)

Can use with `ts` or `group_id` or both. More examples see the other notebook.

In [ ]:
flow_definition6 = {
    'Comment': 'Consume messages to Diaspora Event Fabric',
    'StartAt': 'ConsumeMessages',
    'States': {
        'ConsumeMessages': {
            'Comment': 'Receive messages from a specified topic in Diaspora',
            'Type': 'Action',
            'ActionUrl': action_url,
            'Parameters': {
                'action.$': '$.input.action',
                'topic.$': '$.input.topic',
                'filters.$': '$.input.filters',
            },
            'ResultPath': '$.ConsumeMessages',
            'End': True,
        },
    },
}

flows_client.update_flow(flow_id, definition=flow_definition6, input_schema={})

In [ ]:
flow_input6a = {
    'input': {
        'action': 'consume',
        'topic': topic,
        'filters': [
            {'Pattern': {'value': {'content': [{'prefix': 'hello world1'}]}}},
            {
                'Pattern': {
                    'value': {
                        'content': [
                            {'prefix': 'hello', 'suffix': 'world2'},
                        ],
                    },
                },
            },
        ],
    },
}
print(flow_input6a)

In [ ]:
# Get a client for the flow
specific_flow_authorizer = globus_sdk.AccessTokenAuthorizer(
    access_token=tokens[flow_id]['access_token'],
)
print(tokens[flow_id]['access_token'])
specific_flow_client = globus_sdk.SpecificFlowClient(
    flow_id=flow_id,
    authorizer=specific_flow_authorizer,
)

# Run the flow
# Set a descriptive label for this flow run
run_label = f"Diaspora AP Flow by {primary_identity['preferred_username']}"
run = specific_flow_client.run_flow(
    body=flow_input6a,
    label=run_label,
    tags=['tutorial', 'diaspora'],
)

# Get run details
run_id = run['run_id']
run_status = run['status']
print('This flow can be monitored in the Web App:')
print(f'https://app.globus.org/runs/{run_id}')
print(f'Flow run started with ID: {run_id} - Status: {run_status}')

# Poll the Flow service to check on the status of the flow
while run_status == 'ACTIVE':
    time.sleep(5)
    run = flows_client.get_run(run_id)
    run_status = run['status']
    print(f'Run status: {run_status}')

# Run completed
print(json.dumps(run.data, indent=2))